## Task 2 Week 5

### Classify the CIFAR-10 dataset using Keras deep net, **sequential approach** and **functional API** approach


In [10]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

np.set_printoptions(suppress=True)

In [11]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.cifar10.load_data()
labels=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

In [12]:
print ('x_train samples = ' + str(x_train.shape)) #each image is stored as numpy array 28x28
print ('y_train samples = ' + str(y_train.shape)) #labels

print ('x_test samples = ' + str(x_test.shape))
print ('y_test samples = ' + str(y_test.shape))

x_train samples = (50000, 32, 32, 3)
y_train samples = (50000, 1)
x_test samples = (10000, 32, 32, 3)
y_test samples = (10000, 1)


In [13]:
# preprocess: normalise and re-shape
y_input = tf.keras.utils.to_categorical(y_train)      #one hot encoder
# this line was reshaping to 784 for MNIST as we had 28x28 greyscale images, now it's 32x32x3=3072
# pixel range from 0 to 255 So dividing all the values by 255 will convert it to range from 0 to 1
x_input = (np.reshape(x_train, (x_train.shape[0], 3072)) / 255.0).astype(np.float32)

print ('x_input samples = ' + str(x_input.shape))
print ('y_input samples = ' + str(y_input.shape))

print ('y_input samples = ' + str(y_input))


x_input samples = (50000, 3072)
y_input samples = (50000, 10)
y_input samples = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [14]:
y_valid = tf.keras.utils.to_categorical(y_test)
x_valid = np.reshape(x_test, (x_test.shape[0], 3072)) / 255.0

In [9]:
print ('y_valid shape = ' + str(x_input.shape))
print ('x_valid shape = ' + str(y_input))

y_valid shape = (50000, 3072)
x_valid shape = [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


## Sequential Keras

In [6]:
model = tf.keras.Sequential()
# Adds a densely-connected layer with 512 units to the model:
model.add(tf.keras.layers.Dense(512, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(tf.keras.layers.Dense(10, activation='softmax'))

2023-07-07 14:26:52.959370: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-07 14:26:53.475253: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 198 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0001:00:00.0, compute capability: 3.7
2023-07-07 14:26:53.489378: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 198.44M (208076800 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory


Next, we need to compile the model (this sets the optimizer and the loss function), and then we can train the model.
We set the validation data to be our test dataset so that we can observe the accuracy on the test data during training.


In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_input, y_input, epochs=100, batch_size=256,validation_data=(x_valid, y_valid))

Epoch 1/100
196/196 [==============================] - 1s 4ms/step - loss: 2.0928 - accuracy: 0.2959 - val_loss: 1.9029 - val_accuracy: 0.3241
Epoch 2/100
196/196 [==============================] - 1s 3ms/step - loss: 1.7897 - accuracy: 0.3677 - val_loss: 1.7571 - val_accuracy: 0.3806
Epoch 3/100
196/196 [==============================] - 1s 3ms/step - loss: 1.7163 - accuracy: 0.3958 - val_loss: 1.6953 - val_accuracy: 0.3981
Epoch 4/100
196/196 [==============================] - 1s 3ms/step - loss: 1.6695 - accuracy: 0.4114 - val_loss: 1.6402 - val_accuracy: 0.4182
Epoch 5/100
196/196 [==============================] - 1s 3ms/step - loss: 1.6240 - accuracy: 0.4264 - val_loss: 1.6185 - val_accuracy: 0.4281
Epoch 6/100
196/196 [==============================] - 1s 3ms/step - loss: 1.5797 - accuracy: 0.4424 - val_loss: 1.5842 - val_accuracy: 0.4319
Epoch 7/100
196/196 [==============================] - 1s 3ms/step - loss: 1.5522 - accuracy: 0.4528 - val_loss: 1.5722 - val_accuracy: 0.4413

## Functional API Keras

In [7]:
#We create the input layer and combine with further dense layers,
input_layer = tf.keras.Input(shape=(3072,))
# Add a densely-connected layer with 512 units to the model:
output_layer = tf.keras.layers.Dense(512, activation='relu')(input_layer)

# Add a softmax layer with 10 output units:
output_layer = tf.keras.layers.Dense(10, activation='softmax')(output_layer)

#Note that when input data is one-dimensional, we have to define the shape wih a hanging last dimension (3072,) so
#that it can accommodate an array of input data. Next we construct the model by specifying the inputs and outputs,


Next we construct the model by specifying the inputs and outputs

In [8]:
model = tf.keras.Model(inputs=input_layer,outputs=output_layer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3072)]            0         
                                                                 
 dense_2 (Dense)             (None, 512)               1573376   
                                                                 
 dense_3 (Dense)             (None, 10)                5130      
                                                                 
Total params: 1,578,506
Trainable params: 1,578,506
Non-trainable params: 0
_________________________________________________________________


In [10]:

#The compilation and training then proceeds as per the sequential model,
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_input, y_input, epochs=100, batch_size=256,validation_data=(x_valid, y_valid))

Epoch 1/100
196/196 [==============================] - 2s 6ms/step - loss: 2.0107 - accuracy: 0.3094 - val_loss: 1.7689 - val_accuracy: 0.3835
Epoch 2/100
196/196 [==============================] - 1s 4ms/step - loss: 1.7451 - accuracy: 0.3856 - val_loss: 1.6968 - val_accuracy: 0.4119
Epoch 3/100
196/196 [==============================] - 1s 4ms/step - loss: 1.6722 - accuracy: 0.4118 - val_loss: 1.6524 - val_accuracy: 0.4254
Epoch 4/100
196/196 [==============================] - 1s 4ms/step - loss: 1.6219 - accuracy: 0.4259 - val_loss: 1.5934 - val_accuracy: 0.4420
Epoch 5/100
196/196 [==============================] - 1s 4ms/step - loss: 1.5810 - accuracy: 0.4443 - val_loss: 1.5772 - val_accuracy: 0.4370
Epoch 6/100
196/196 [==============================] - 1s 4ms/step - loss: 1.5554 - accuracy: 0.4538 - val_loss: 1.5782 - val_accuracy: 0.4415
Epoch 7/100
196/196 [==============================] - 1s 4ms/step - loss: 1.5344 - accuracy: 0.4615 - val_loss: 1.5749 - val_accuracy: 0.4405